In [5]:
import torch
from torch.autograd import Variable
import numpy as np

In [6]:
test_x=torch.linspace(0,100,101,dtype=torch.float64).reshape(101,1)

In [7]:
train_x=torch.tensor([3,1,5,8,9,2],dtype=torch.float64).reshape(6,1)

In [8]:
train_y=torch.cos(train_x)+torch.normal(0,0.3,(6,1))

In [9]:
class GPR(object):
    def __init__(self,train_x,train_y,test_x,sigma,L):
        self.train_x=train_x
        self.train_y=train_y
        self.test_x=test_x
        self.sigma=sigma
        self.L=L
    
    def kernel(self,x1,x2):
        x=x1.reshape(-1,1)**2+x2.reshape(1,-1)**2-2*torch.mm(x1,x2.T)
        return self.sigma**2*torch.exp(-(x)/(2*(self.L**2)))
    
    def get_mu_cov_loss(self):
        Kff=self.kernel(self.train_x,train_x) #（N,N）
        Kfy=self.kernel(self.train_x,self.test_x) #（N,K）
        Kyf=Kfy.T#(K,N)
        Kyy=self.kernel(self.test_x,self.test_x)#(k,k)
        
        mu=torch.mm(torch.mm(Kyf,Kff.inverse()),self.train_y)
        cov=Kyy-torch.mm(torch.mm(Kyf,Kff.inverse()),Kfy)
        loss=1/2*torch.mm(torch.mm(self.train_y.T,Kff.inverse()),self.train_y)+1/2*torch.log(torch.det(Kff))
        
        return mu,cov,loss
    

In [13]:
def train(epoches,train_x,train_y,test_x):
    sigma=Variable(torch.randn(1,1,dtype=torch.float64),requires_grad=True)
    L=Variable(torch.randn(1,1,dtype=torch.float64),requires_grad=True)
    learn_rating=0.0001
    for i in range(epoches):
        mu,cov,loss=GPR(train_x,train_y,test_x,sigma,L).get_mu_cov_loss()
        loss.backward()
        sigma.data=sigma.data-learn_rating*sigma.grad.data
        L.data=L.data-learn_rating*L.grad.data
        
        sigma.grad.data.zero_()
        L.grad.data.zero_()
        
        
    mu,cov,loss=GPR(train_x,train_y,test_x,sigma,L).get_mu_cov_loss()
    return mu


In [14]:
mu=train(100,train_x,train_y,test_x)